In [8]:
import numpy as np
import pandas as pd

In [9]:
#get all the data from the csv file
df = pd.read_csv('../dataset/breast-cancer-wisconsin.data', header=None)
df.replace('?', 5, inplace=True)
df.drop([0], axis=1, inplace=True)
df = df
df

,1,2,3,4,5,6,7,8,9,10
0,5,1,1,1,2,1,3,1,1,2
1,5,4,4,5,7,10,3,2,1,2
2,3,1,1,1,2,2,3,1,1,2
3,6,8,8,1,3,4,3,7,1,2
4,4,1,1,3,2,1,3,1,1,2
...,...,...,...,...,...,...,...,...,...,...
694,3,1,1,1,3,2,1,1,1,2
695,2,1,1,1,2,1,1,1,1,2
696,5,10,10,3,7,3,8,10,2,4
697,4,8,6,4,3,4,10,6,1,4


In [12]:
#shuffle the data
df = df.sample(frac=1)
#split the data into training and testing
x = np.array(df.drop([10], axis=1),int)
y = np.array(df[10],int)
test_size = int(len(x) * 0.3)
print(x.shape, y.shape, test_size)
#split the data into training and testing
x_test = x[:test_size]
y_test = y[:test_size]

x_train = x[test_size:]
y_train = y[test_size:]

print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)


(699, 9) (699,) 209
(490, 9) (490,) (209, 9) (209,)


In [13]:
#estimate P(Y)
unique, counts = np.unique(y_train, return_counts=True)
print(unique, counts)
#finding P(Y=2) and P(Y=4)
p_y2 = counts[0]/y_train.shape[0]
p_y4 = counts[1]/y_train.shape[0]
print("P(Y=2) = ", p_y2, "P(Y=4) = ", p_y4)


[2 4] [326 164]
P(Y=2) =  0.6653061224489796 P(Y=4) =  0.3346938775510204


In [14]:
#estimate P(X|Y)
px_y2 = np.zeros((9,10))
px_y4 = np.zeros((9,10))
#estimate P(X|Y=2)
for i in range(0,9):
    unique, counts = np.unique(x_train[y_train==2,i], return_counts=True)
    px_y2[i,unique-1] = counts/np.sum(counts)
#estimate P(X|Y=4)
for i in range(0,9):
    unique, counts = np.unique(x_train[y_train==4,i], return_counts=True)
    px_y4[i,unique-1] = counts/np.sum(counts)
    # print(unique, counts)
    # print(px_y4[i,unique-1])

In [15]:
#calculate theta
theta_y2 = np.zeros((9,10))
theta_y4 = np.zeros((9,10))
alpha = 0 #smoothing parameter
for i in range(0,9):
    theta_y2[i] = (px_y2[i]+alpha)/(np.sum(px_y2[i])+alpha*10)
    print(theta_y2[i])
    theta_y4[i] = (px_y4[i]+alpha)/(np.sum(px_y4[i])+alpha*10)

[0.30981595 0.10736196 0.21779141 0.13496933 0.18404908 0.03374233
 0.00306748 0.00920245 0.         0.        ]
[0.8404908  0.07668712 0.05214724 0.01840491 0.         0.00613497
 0.         0.00306748 0.00306748 0.        ]
[0.77607362 0.10429448 0.06748466 0.03067485 0.00613497 0.00920245
 0.00306748 0.00306748 0.         0.        ]
[0.81595092 0.07668712 0.06748466 0.01226994 0.00920245 0.01226994
 0.         0.         0.00306748 0.00306748]
[0.11042945 0.78220859 0.06441718 0.01533742 0.00613497 0.00613497
 0.00920245 0.00306748 0.         0.00306748]
[0.82822086 0.04907975 0.03374233 0.01840491 0.05828221 0.
 0.00306748 0.00306748 0.         0.00613497]
[0.33742331 0.3404908  0.27607362 0.01533742 0.01226994 0.00306748
 0.01533742 0.         0.         0.        ]
[0.88343558 0.06441718 0.02760736 0.         0.00306748 0.00306748
 0.00306748 0.01226994 0.00306748 0.        ]
[0.96625767 0.02147239 0.00306748 0.         0.00306748 0.
 0.00306748 0.00306748 0.         0.        ]

In [18]:
#testing
def predict(x):
    p2 = p_y2
    p4 = p_y4
    for i in range(0,9):
        p2 = p2*theta_y2[i,x[i]-1]
        p4 = p4*theta_y4[i,x[i]-1]
    if p2>p4:
        return 2
    else:
        return 4


In [19]:
y_pred = np.zeros(y_test.shape)
for i in range(0,y_test.shape[0]):
    y_pred[i] = predict(x_test[i])

#calculate accuracy
accuracy = np.sum(y_pred==y_test)/y_test.shape[0]
print("accuracy = ", accuracy)
#calculate error
error = np.sum(y_pred!=y_test)
print("error = ", error, "total = ", y_test.shape[0])

accuracy =  0.9808612440191388
error =  4 total =  209
